In [83]:
#import libraries 
import numpy as np  
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
#from imblearn.over_sampling import RandomOverSampler 
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import RobustScaler 

In [84]:
#import data 
white_wine = pd.read_csv("wine+quality\winequality-white.csv", sep= ';', delimiter=None) 

#make a copy
ww = white_wine

In [85]:
#remove all duplicates 
ww.drop_duplicates(inplace=True)

In [86]:
#unify column names 
ww.rename(columns = {'fixed acidity':'fixed_acidity','volatile acidity':'volatile_acidity','citric acid':'citric_acid','residual sugar':'residual_sugar','free sulfur dioxide':'free_sulfur_dioxide', 'total sulfur dioxide':'total_sulfur_dioxide'}, inplace=True)

In [87]:
#add a column 
#ww.insert(12, "colour", 'white')

Next add the quality label column 

In [88]:
#add quality label column - low 1-4, medium 5-6, high 7-10 
ww['quality_label'] = ww['quality'].apply(lambda value: 'low'
if value <= 4 else 'medium'
if value <= 6 else 'high')
ww['quality_label'] = pd.Categorical(ww['quality_label'], 
categories=['low', 'medium', 'high']) 

Label Encoder 

In [89]:
ww1 = ww.copy()

In [90]:
#check unique values of this column 
ww1['quality_label'].unique()

['medium', 'high', 'low']
Categories (3, object): ['low', 'medium', 'high']

In [91]:
#first save label encoder 
lbl_enc = LabelEncoder()

#use function fit transform 
ww1['quality_label'] = lbl_enc.fit_transform(ww1['quality_label'])


In [92]:
ww['quality_label'].value_counts()

medium    2963
high       825
low        173
Name: quality_label, dtype: int64

In [93]:
ww1['quality_label'].value_counts()
# low = 1 
# medium = 2 
# high = 0 

2    2963
0     825
1     173
Name: quality_label, dtype: int64

In [94]:
#for label in ww.columns[:-1]:
    #plt.hist(ww[ww["quality_label"]==0][label], color = 'blue', label = 'High Quality', alpha = 0.7, density=True)
    #plt.hist(ww[ww["quality_label"]==1][label], color = 'red', label = 'Low Quality', alpha = 0.7, density=True)
    #plt.title(label)
    #plt.xlabel(label)
    #plt.legend()
    #plt.show()

In [95]:
#manually calculate interquartile range to compare to robust scaler 
#too far into the math 
#q25, q50, q75 = np.percentile(ww, [25, 50, 75], axis=1)
#x_scaled = (ww-q50)/(ww-q75-q25)
#print(x_scaled)

In [96]:
#split data
train, valid, test = np.split(ww1.sample(frac=1), [int(0.6*len(ww1)), int(0.8*len(ww1))])

In [97]:
#robust scaler example 
#scaler = RobustScaler()
#ww_scaled = scaler.fit_transform(ww) 
#print(ww_scaled)

In [98]:
#robust scaler in code

#def scale_dataset(ww1, oversample=False):
 #   X = ww1[ww1.columns[:-1]].values #calling all columns until the last one 
  #  y = ww1[ww1.columns[-1]].values #calling only the last column 
   # 
    #scaler = RobustScaler()
    #X = scaler.fit_transform(ww1) 

#    data = np.hstack((X, np.reshape(y, (-1, 1))))
 #   # y is only 2D, so call numpy reshape to make it  3Dimensional Item 
  #  # because y is only 1 column
#
 #   return data, X, y


In [99]:
#youtube explaination of numpy arrawy dimensions and hstack in machine learning 

In [100]:
# standard scaler
#scale data 
def scale_dataset(ww1, oversample=False):
    X = ww1[ww1.columns[:-1]].values #calling all columns until the last one 
    y = ww1[ww1.columns[-1]].values #calling only the last column 

    scaler = StandardScaler() # check rules here for group splitting 
    X = scaler.fit_transform(X) 

    data = np.hstack((X, np.reshape(y, (-1, 1))))
    # y is only 2D, so call numpy reshape to make it  3Dimensional Item 
    # because y is only 1 column

    return data, X, y

In [101]:
#train data and run model 
train, X_train, y_train = scale_dataset(train)
valid, X_valid, y_valid = scale_dataset(valid)
test, X_test, y_test = scale_dataset(test)

knn_model = KNeighborsClassifier(n_neighbors=7)
knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)


In [102]:
#check classification report 
print(classification_report(y_test, y_pred)) 

# low is 1 
# medium is 2 
# high is 0

              precision    recall  f1-score   support

           0       0.98      0.91      0.95       151
           1       1.00      0.26      0.41        31
           2       0.94      1.00      0.97       611

    accuracy                           0.95       793
   macro avg       0.97      0.72      0.77       793
weighted avg       0.95      0.95      0.94       793



In [103]:
print(confusion_matrix(y_test, y_pred))

[[138   0  13]
 [  0   8  23]
 [  3   0 608]]
